Initialisation 

In [3]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [8]:
# chargement du dataset
df = pd.read_csv('../data/events.csv')
df.head()
df.info()
df['event'].value_counts(normalize=False)
df['event'].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64

Nettoyage et exploration 

In [9]:
# filter uniquement vieuw et add_to_cart
df_ab = df[df['event'].isin(['view', 'add_to_cart'])].copy

In [11]:
# valeurs manquantes
print(df.isna().sum())

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64


In [12]:
# nombre d'evenements par type dans ce sous-ensemble
print(df["event"].value_counts())
print(df["event"].value_counts(normalize=True))

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64
event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64


In [13]:
# nomnre d'users uniques
n_visiteurs = df["visitorid"].nunique()
print("Visiteurs uniques :", n_visiteurs)

Visiteurs uniques : 1407580


Simulation de l’A/B test

In [14]:
# visiteurs unique 
visitors = df["visitorid"].drop_duplicates()
len(visitors)

1407580

In [15]:
#  Assigner A/B aléatoirement (50/50)
np.random.seed(42)  # Pour la reproductibilité
assignments = np.random.choice(["A", "B"], size=len(visitors), p=[0.5, 0.5])

visitor_groups = pd.DataFrame({
    "visitorid": visitors.values,
    "group": assignments
})

visitor_groups["group"].value_counts(normalize=True)


group
A    0.500024
B    0.499976
Name: proportion, dtype: float64